In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import pandas as pd

# Load the data labels (subclasses and superclasses)

In [2]:
# Load mappings
superclass_df = pd.read_csv('/content/superclass_mapping.csv')
subclass_df = pd.read_csv('/content/subclass_mapping.csv')
num_superclasses = len(superclass_df)
num_subclasses = len(subclass_df)

get_superclass = dict(zip(superclass_df['index'], superclass_df['class']))
get_subclass = dict(zip(subclass_df['index'], subclass_df['class']))

# Setting up the Training and Validation Datasets

In [ ]:
# Load the data
!unzip '/content/train_images.zip'
!unzip '/content/test_images.zip'

In [4]:
# Load annotation CSV
ann_df = pd.read_csv('/content/train_data.csv')

# Images path
img_dir = '/content/train_images/'

# Split into training (90%) and validation (10%) sets
train_df, val_df = train_test_split(ann_df, test_size=0.1, random_state=42, shuffle=True)

IMAGE_SHAPE = [380, 380] # Because of EfficientNetB4

# Pre-Processing WITHOUT AUGMENTATION
def process_image_no_aug(img_path, super_label, sub_label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE)

    # Create one-hot encodings
    super_label = tf.one_hot(super_label, depth=num_superclasses)
    sub_label = tf.one_hot(sub_label, depth=num_subclasses)

    return img, {'super_output': super_label, 'sub_output': sub_label}

# Pre-Processing WITH AUGMENTATION
def process_image_with_aug(img_path, super_label, sub_label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE)

    # Data augmentation
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    img = tf.image.random_saturation(img, lower=0.95, upper=1.05)
    img = tf.image.random_hue(img, max_delta=0.02)

    # Create one-hot encodings
    super_label = tf.one_hot(super_label, depth=num_superclasses)
    sub_label = tf.one_hot(sub_label, depth=num_subclasses)

    return img, {'super_output': super_label, 'sub_output': sub_label}

BATCH_SIZE = 32

def get_image_paths(df, base_dir):
  image_paths = []
  for filename in df['image']:
    image_paths.append(os.path.join(base_dir, filename))
  return image_paths

# ====== Creating the training dataset =========
image_paths = get_image_paths(train_df, '/content/train_images')
dataset = tf.data.Dataset.from_tensor_slices((image_paths, train_df['superclass_index'].values, train_df['subclass_index'].values))
# Data augmentation
dataset = dataset.map(process_image_with_aug, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ====== Creating the validation dataset =========
image_paths = get_image_paths(val_df, '/content/train_images')
dataset = tf.data.Dataset.from_tensor_slices((image_paths, val_df['superclass_index'].values, val_df['subclass_index'].values))
# No data augmentation
dataset = dataset.map(process_image_no_aug, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Setting up the Test Dataset

In [5]:
# ====== Creating the test dataset =========
test_image_filenames = sorted([f for f in os.listdir('/content/test_images/')])
test_image_paths = [os.path.join('/content/test_images/', fname) for fname in test_image_filenames]

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_paths, test_image_filenames))

def process_test_image(img_path, img_name):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE)
    return img, img_name

test_dataset = test_dataset.map(process_test_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Calculating the Class Imbalance in the Training Dataset

In [ ]:
# This code to plot the class imbalances was generated by ChatGPT.

# Subclass imbalance

# How many times each subclass appears, sorted by subclass index (not by descending count).
subclass_counts = train_df['subclass_index'].value_counts().sort_index()

print("Subclass Distribution:")
print(subclass_counts)

# Plot
plt.figure(figsize=(16, 4))
subclass_counts.plot(kind='bar')
plt.title('Subclass Distribution')
plt.xlabel('Subclass Index')
plt.ylabel('Count')
plt.show()

#==========

# Superclass imbalance

# How many times each superclass appears, sorted by superclass index (not by descending count).
superclass_counts = train_df['superclass_index'].value_counts().sort_index()

print("Superclass Distribution:")
print(superclass_counts)

# Plot
plt.figure(figsize=(6, 4))
superclass_counts.plot(kind='bar', color='orange')
plt.title('Superclass Distribution')
plt.xlabel('Superclass Index')
plt.ylabel('Count')
plt.show()

# Calculating the Class Imbalance in the Validation Dataset

In [ ]:
# This code to plot the class imbalances was generated by ChatGPT.

# Subclass imbalance

# How many times each subclass appears, sorted by subclass index (not by descending count).
subclass_counts = val_df['subclass_index'].value_counts().sort_index()

print("Subclass Distribution:")
print(subclass_counts)

# Plot
plt.figure(figsize=(16, 4))
subclass_counts.plot(kind='bar')
plt.title('Subclass Distribution')
plt.xlabel('Subclass Index')
plt.ylabel('Count')
plt.show()

#==========

# Superclass imbalance

# How many times each superclass appears, sorted by superclass index (not by descending count).
superclass_counts = val_df['superclass_index'].value_counts().sort_index()

print("Superclass Distribution:")
print(superclass_counts)

# Plot
plt.figure(figsize=(6, 4))
superclass_counts.plot(kind='bar', color='orange')
plt.title('Superclass Distribution')
plt.xlabel('Superclass Index')
plt.ylabel('Count')
plt.show()

# Implement the Class-Weighted Cross-Entropy Loss

In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf

subclass_labels = ann_df['subclass_index'].values

# Compute weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(subclass_labels), y=subclass_labels)

# Convert to Tensor
subclass_class_weights = tf.constant(class_weights_array, dtype=tf.float32)

# Dummy weight for the "novel" subclass category
novel_weight = np.array([1.0], dtype=np.float32)
subclass_class_weights = tf.constant(np.concatenate([class_weights_array, novel_weight]), dtype=tf.float32)

In [8]:
import tensorflow.keras.backend as K

def class_weighted_loss(weights):
    # Assign a weight to each class
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # Avoid log(0) via clipping
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, axis=-1)
        return loss

    return loss

subclass_loss_fn = class_weighted_loss(subclass_class_weights)

# Load EfficientNetB4 as the feature extractor

In [ ]:
def build_dual_head_model(input_shape=(380, 380, 3)):
    # Load EfficientNetB4 base model
    base_model = EfficientNetB4(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze base model

    # Shared feature extraction
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Add 2 classification heads (including the "novel" super/sub class in both)
    super_output = Dense(num_superclasses, activation='softmax', name='super_output')(x)
    sub_output = Dense(num_subclasses, activation='softmax', name='sub_output')(x)

    model = Model(inputs=base_model.input, outputs=[super_output, sub_output])
    return model

model = build_dual_head_model()

In [10]:
# Compile the model with the loss function
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        'super_output': 'categorical_crossentropy',  # normal loss
        'sub_output': subclass_loss_fn              # weighted loss
    },
    loss_weights={
        'super_output': 1.0,
        'sub_output': 1.0
    },
    metrics={
        'super_output': 'accuracy',
        'sub_output': 'accuracy'
    }
)

# Train the Baseline and get results!

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_weighted_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=callbacks
)


# Checking the performance of the Baseline + Weighted Loss Model

In [ ]:
# The code below was written with the help of ChatGPT to evaluate the performance of my model.

import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Rebuild and load the model
model = build_dual_head_model()
model.load_weights('best_weighted_model.h5')

# Compile the model using the same settings as training
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        'super_output': 'categorical_crossentropy',  # normal loss
        'sub_output': subclass_loss_fn              # class-weighted cross-entropy loss
    },
    metrics={
        'super_output': 'accuracy',
        'sub_output': 'accuracy'
    }
)

# Evaluate the model on the validation set
val_results = model.evaluate(val_dataset)
print("\n🔹 Validation Results:")
for name, value in zip(model.metrics_names, val_results):
    print(f"{name}: {value:.4f}")

# Get the predictions and true labels
true_super, pred_super = [], []
true_sub, pred_sub = [], []

for batch in val_dataset:
    images, labels = batch
    preds = model.predict(images, verbose=0)

    # True labels
    true_super += np.argmax(labels['super_output'].numpy(), axis=1).tolist()
    true_sub += np.argmax(labels['sub_output'].numpy(), axis=1).tolist()

    # Predicted labels
    pred_super += np.argmax(preds[0], axis=1).tolist()
    pred_sub += np.argmax(preds[1], axis=1).tolist()

# Use the predictions and true labels to construct CONFUSION MATRICES
super_cm = confusion_matrix(true_super, pred_super)
sub_cm = confusion_matrix(true_sub, pred_sub)

# Convert to NumPy array if needed
sub_cm = np.array(sub_cm)

# Convert to DataFrame for easier indexing
sub_df = pd.DataFrame(sub_cm)

# Zero out the diagonal (correct predictions) so we only see misclassifications
sub_df_no_diag = sub_df.copy()
np.fill_diagonal(sub_df_no_diag.values, 0)

# Find the most confused class pairs (top N)
N = 5
confused_pairs = sub_df_no_diag.stack().sort_values(ascending=False).head(N)

print("\n🔍 Top Confused Subclass Pairs:")
for (true_class, pred_class), count in confused_pairs.items():
    print(f"True class {true_class} → Predicted class {pred_class}: {count} times")

# Plot Superclass Confusion Matrix
plt.figure(figsize=(6,5))
sns.heatmap(super_cm, annot=True, fmt='d', cmap='Blues')
plt.title("Superclass Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Plot Subclass Confusion Matrix
plt.figure(figsize=(12,10))
sns.heatmap(sub_cm, cmap='Blues')
plt.title("Subclass Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# F1 scores
print("\n🔹 Superclass Classification Report:")
print(classification_report(true_super, pred_super))

print("\n🔹 Subclass Classification Report:")
print(classification_report(true_sub, pred_sub))

# Fine-Tuning the Model (only the top few layers starting from 'block6h')

In [12]:
model = build_dual_head_model()
model.load_weights('best_weighted_model.h5')

# Starting freezing from block6h
unfreeze = False
for layer in model.layers:
    if 'block6h' in layer.name:
        unfreeze = True
    layer.trainable = unfreeze

# Compiling the class-weighted-loss model with a SLOWER learning rate (for fine-tuning)
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss={
        'super_output': 'categorical_crossentropy',  # normal loss
        'sub_output': subclass_loss_fn              # weighted loss
    },
    metrics={
        'super_output': 'accuracy',
        'sub_output': 'accuracy'
    }
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_weighted_finetuned_model_6h.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=callbacks
)

# Checking the performance of the Fine-Tuned Model

In [ ]:
# The code below was generated by ChatGPT to help me evaluate the performance of my model.

import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Rebuild and load the fine-tuned model
model = build_dual_head_model()
model.load_weights('best_weighted_finetuned_model_6h.h5')

# Compile the model using the same settings as training
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        'super_output': 'categorical_crossentropy',  # normal loss
        'sub_output': subclass_loss_fn              # class-weighted cross-entropy loss
    },
    metrics={
        'super_output': 'accuracy',
        'sub_output': 'accuracy'
    }
)

# Evaluate the model on the validation set
val_results = model.evaluate(val_dataset)
print("\n🔹 Validation Results:")
for name, value in zip(model.metrics_names, val_results):
    print(f"{name}: {value:.4f}")

# Get the predictions and true labels
true_super, pred_super = [], []
true_sub, pred_sub = [], []

for batch in val_dataset:
    images, labels = batch
    preds = model.predict(images, verbose=0)

    # True labels
    true_super += np.argmax(labels['super_output'].numpy(), axis=1).tolist()
    true_sub += np.argmax(labels['sub_output'].numpy(), axis=1).tolist()

    # Predicted labels
    pred_super += np.argmax(preds[0], axis=1).tolist()
    pred_sub += np.argmax(preds[1], axis=1).tolist()

# Use the predictions and true labels to create CONFUSION MATRICES
super_cm = confusion_matrix(true_super, pred_super)
sub_cm = confusion_matrix(true_sub, pred_sub)

# Plot Superclass Confusion Matrix
plt.figure(figsize=(6,5))
sns.heatmap(super_cm, annot=True, fmt='d', cmap='Blues')
plt.title("Superclass Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Plot Subclass Confusion Matrix
plt.figure(figsize=(12,10))
sns.heatmap(sub_cm, cmap='Blues')
plt.title("Subclass Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# F1 Scores
print("\n🔹 Superclass Classification Report:")
print(classification_report(true_super, pred_super))

print("\n🔹 Subclass Classification Report:")
print(classification_report(true_sub, pred_sub))

# Generating the test predictions for the leaderboard

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd

test_predictions = {
    'image': [],
    'superclass_index': [],
    'subclass_index': []
}

# Loop through the test dataset and make predictions
for batch in test_dataset:  # Each batch contains (images, image_names)
    images, image_names = batch
    preds = model.predict(images, verbose=0)

    super_preds = np.argmax(preds[0], axis=1)
    sub_preds = np.argmax(preds[1], axis=1)

    for i in range(len(image_names)):
        filename = image_names[i].numpy().decode("utf-8")
        test_predictions['image'].append(filename)
        test_predictions['superclass_index'].append(int(super_preds[i]))
        test_predictions['subclass_index'].append(int(sub_preds[i]))

# Save predictions to CSV
df = pd.DataFrame(test_predictions)
df.to_csv("test_predictions.csv", index=False)
print("Test predictions SAVED")

Test predictions SAVED
